In [ ]:
!pip install google-cloud-aiplatform



In [2]:
from google.colab import userdata
from google.cloud import aiplatform  # Vertex AI's library

# Configure the API client with your Google API Key
api_key = userdata.get("GOOGLE_API_KEY")
aiplatform.init(project="your-project-id", location="us-central1", api_endpoint="us-central1-aiplatform.googleapis.com")


In [3]:
import google.generativeai as genai
from google.colab import userdata

# Configure your API key (ensure you've set it in userdata or manually here)
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

# Define the fan control functions as before
def enable_fan():
    """Turn on the fan system."""
    print("FANBOT: Fan enabled.")

def set_fan_speed(speed: str):
    """Set the fan speed. Fan must be enabled for this to work."""
    print(f"FANBOT: Fan speed set to {speed}.")

def stop_fan():
    """Stop the fan system."""
    print("FANBOT: Fan turned off.")

# List of tools (functions)
fan_controls = [enable_fan, set_fan_speed, stop_fan]

# System instruction for the AI model
instruction = "You are a helpful fan system bot. You can turn the fan on and off, and you can set the speed. Do not perform any other tasks."

# Initialize the model (Ensure this is done correctly with a proper model ID)
model = genai.GenerativeModel(
    "models/gemini-1.5-pro", tools=fan_controls, system_instruction=instruction
)

# Start the chat session to get the 'chat' object
chat = model.start_chat()  # This is important: the 'chat' object should be initialized here


In [4]:
from google.cloud.aiplatform import gapic as aiplatform_gapic
from collections.abc import Iterable

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return {
        "function_calling_config": {
            "mode": mode,
            "allowed_function_names": fns
        }
    }


In [5]:
# Define the tool config as before
def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return {
        "function_calling_config": {
            "mode": mode,
            "allowed_function_names": fns
        }
    }

# Tool config for mode 'none' (no function calls will be made)
tool_config = tool_config_from_mode("none")

# Send a message to the model to ask about its capabilities
response = chat.send_message(
    "Hello fan-bot, what can you do?", tool_config=tool_config
)

# Print the response text from the model
print(response.text)


I can turn the fan on and off, and I can set the speed.



In [6]:
tool_config = tool_config_from_mode("auto")

# Send a message that will trigger automatic function calls (like turning on the fan)
response = chat.send_message("Turn on the fan!", tool_config=tool_config)

# Print the response and verify the function was called
print(response.parts[0])

# Reset the chat history to avoid unnecessary function calls
chat.rewind()


function_call {
  name: "enable_fan"
  args {
  }
}



(parts {
   text: "Turn on the fan!"
 }
 role: "user",
 parts {
   function_call {
     name: "enable_fan"
     args {
     }
   }
 }
 role: "model")

In [7]:
# List of available functions for this message
available_fns = ["set_fan_speed", "stop_fan"]

# Tool configuration to allow specific functions
tool_config = tool_config_from_mode("any", available_fns)

# Send a message to set the fan speed to high
response = chat.send_message("Set the fan speed to HIGH!", tool_config=tool_config)

# Print the response from the model
print(response.parts[0])


function_call {
  name: "set_fan_speed"
  args {
    fields {
      key: "speed"
      value {
        string_value: "HIGH"
      }
    }
  }
}



In [8]:
import time

def send_message_with_retry(auto_chat, message, tool_config, max_retries=3):
    for i in range(max_retries):
        try:
            return auto_chat.send_message(message, tool_config=tool_config)
        except genai.TooManyRequests:
            if i < max_retries - 1:
                sleep_time = 2 ** i  # Exponential backoff
                print(f"Rate limit exceeded. Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                raise

# Retry sending the message
send_message_with_retry(chat, "It's awful dark in here...", tool_config)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "set_fan_speed",
                  "args": {
                    "speed": "HIGH"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.06310672419411796
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 196,
        "candidates_token_count": 7,
        "total_token_count": 203
      }
    }),
)

In [9]:
import datetime
import time

# Function to simulate turning off the fan
def stop_fan():
    print("FANBOT: The fan has been turned OFF.")

def turn_off_fan_in_morning():
    """Automatically turns off the fan at 6:00 AM."""
    while True:
        current_time = datetime.datetime.now()
        # Check if it's 6:00 AM
        if current_time.hour == 6 and current_time.minute == 0:
            stop_fan()  # Turn off the fan
            print("FANBOT: It's 6:00 AM, turning off the fan automatically.")
            time.sleep(60)  # Sleep for a minute to avoid repeating it multiple times in the same minute
        time.sleep(10)  # Check every 10 seconds if it's time to turn off the fan

# Run the function in a separate thread for continuous checking
import threading
threading.Thread(target=turn_off_fan_in_morning, daemon=True).start()


In [10]:
import datetime
import time

# Function to simulate turning off the fan
def stop_fan():
    print("FANBOT: The fan has been turned OFF.")

def turn_off_fan_in_morning():
    """Automatically turns off the fan at 6:00 AM."""
    current_time = datetime.datetime.now()
    print(f"Current Time: {current_time.strftime('%H:%M:%S')}")

    if current_time.hour == 6 and current_time.minute == 0:
        stop_fan()  # Turn off the fan
        print("FANBOT: It's 6:00 AM, turning off the fan automatically.")
    else:
        print("FANBOT: It's not 6:00 AM yet, checking again soon.")

# Run the function once to immediately get an output (no continuous checking)
turn_off_fan_in_morning()


Current Time: 07:00:39
FANBOT: It's not 6:00 AM yet, checking again soon.
